In [2]:
import numpy as np
import statsmodels.api as sm
from statsmodels.regression.mixed_linear_model import VCSpec
import pandas as pd

In [3]:
np.random.seed(3123)

### Nested analysis

In [4]:
def generate_nested(n_group1=200, n_group2=20, n_rep=10, group1_sd=2, group2_sd=3, unexplained_sd=4):

    # Group 1 indicators
    group1 = np.kron(np.arange(n_group1), np.ones(n_group2 * n_rep))

    # Group 1 effects
    u = group1_sd * np.random.normal(size=n_group1)
    effects1 = np.kron(u, np.ones(n_group2 * n_rep))

    # Group 2 indicators
    group2 = np.kron(np.ones(n_group1), np.kron(np.arange(n_group2), np.ones(n_rep)))

    # Group 2 effects
    u = group2_sd * np.random.normal(size=n_group1 * n_group2)
    effects2 = np.kron(u, np.ones(n_rep))

    e = unexplained_sd * np.random.normal(size=n_group1 * n_group2 * n_rep)
    y = effects1 + effects2 + e

    df = pd.DataFrame({"y": y, "group1": group1, "group2": group2})

    return df

df = generate_nested()
df

,y,group1,group2
0,-2.193553,0.0,0.0
1,-1.996399,0.0,0.0
2,-7.268743,0.0,0.0
3,-4.701142,0.0,0.0
4,3.464830,0.0,0.0
...,...,...,...
39995,-1.709221,199.0,19.0
39996,10.295354,199.0,19.0
39997,1.312228,199.0,19.0
39998,-0.339235,199.0,19.0


In [5]:
model1 = sm.MixedLM.from_formula(
    'y ~ 1',
    re_formula='1',
    vc_formula={'group2': '0 + C(group2)'},
    groups='group1',
    data=df,
)
result1 = model1.fit()
print(result1.summary())

          Mixed Linear Model Regression Results
Model:            MixedLM Dependent Variable: y           
No. Observations: 40000   Method:             REML        
No. Groups:       200     Scale:              15.8825     
Min. group size:  200     Log-Likelihood:     -116022.3805
Max. group size:  200     Converged:          Yes         
Mean group size:  200.0                                   
-----------------------------------------------------------
            Coef.   Std.Err.    z     P>|z|  [0.025  0.975]
-----------------------------------------------------------
Intercept   -0.035     0.149  -0.232  0.817  -0.326   0.257
group1 Var   3.917     0.112                               
group2 Var   8.742     0.063                               



### Crossed analysis

In [6]:
def generate_crossed(
    n_group1=100, n_group2=100, n_rep=4, group1_sd=2, group2_sd=3, unexplained_sd=4
):

    # Group 1 indicators
    group1 = np.kron(
        np.arange(n_group1, dtype=int), np.ones(n_group2 * n_rep, dtype=int)
    )
    group1 = group1[np.random.permutation(len(group1))]

    # Group 1 effects
    u = group1_sd * np.random.normal(size=n_group1)
    effects1 = u[group1]

    # Group 2 indicators
    group2 = np.kron(
        np.arange(n_group2, dtype=int), np.ones(n_group2 * n_rep, dtype=int)
    )
    group2 = group2[np.random.permutation(len(group2))]

    # Group 2 effects
    u = group2_sd * np.random.normal(size=n_group2)
    effects2 = u[group2]

    e = unexplained_sd * np.random.normal(size=n_group1 * n_group2 * n_rep)
    y = effects1 + effects2 + e

    df = pd.DataFrame({"y": y, "group1": group1, "group2": group2})

    return df

df = generate_crossed()
df

,y,group1,group2
0,7.064740,41,19
1,-12.702945,11,57
2,-2.359046,21,36
3,3.755098,92,48
4,-4.464436,35,98
...,...,...,...
39995,-1.295849,97,4
39996,-12.690511,7,80
39997,4.978047,0,65
39998,1.154263,36,41


In [7]:
vc = {
    'g1': '0 + C(group1)', 
    'g2': '0 + C(group2)'
}
oo = np.ones(df.shape[0])
model3 = sm.MixedLM.from_formula(
    'y ~ 1', 
    groups=oo,
    vc_formula=vc, 
    data=df
)
result3 = model3.fit()
print(result3.summary())

          Mixed Linear Model Regression Results
Model:            MixedLM Dependent Variable: y           
No. Observations: 40000   Method:             REML        
No. Groups:       1       Scale:              15.9824     
Min. group size:  40000   Log-Likelihood:     -112684.9688
Max. group size:  40000   Converged:          Yes         
Mean group size:  40000.0                                 
-----------------------------------------------------------
            Coef.   Std.Err.    z     P>|z|  [0.025  0.975]
-----------------------------------------------------------
Intercept   -0.251     0.353  -0.710  0.478  -0.943   0.442
g1 Var       4.282     0.154                               
g2 Var       8.150     0.291                               

